# BoxBunny Unit Tests
Independent unit tests for system components and integration tests.

## Setup Environment
Run this first to configure the shell environment.

In [ ]:
%%bash
cat > /tmp/boxbunny_env.sh <<'SH'
source ~/miniconda3/etc/profile.d/conda.sh
conda activate boxing_ai
export PYTHONNOUSERSITE=1
export LD_LIBRARY_PATH=$CONDA_PREFIX/lib:$LD_LIBRARY_PATH
export SETUPTOOLS_USE_DISTUTILS=stdlib
source /opt/ros/humble/setup.bash
source /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/setup.bash 2>/dev/null || true
SH
echo "Environment ready"

## Build Workspace
**Critical:** Run this to build `boxbunny_msgs` and other packages.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
colcon build --symlink-install
source install/setup.bash

## 1. Unit: GUI Layout
Test the GUI appearance and layout **without** backend services.
**Expectation:** GUI opens. Buttons like "Start Drill" will log warnings ("service not ready") but not crash.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/src/boxbunny_gui/boxbunny_gui
python gui_main.py

## 2. Unit: Camera & AI (Pose)
Test the **Action Model** (Pose/YOLO) for advanced detection.
**Expectation:** Camera window opens with skeleton tracking debug overlay.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/action_prediction
python tools/inference/live_infer_rgbd.py \
    --model-config configs/rgbd_boxing_anticipation.py \
    --model-checkpoint /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/models/action_prediction_model/best_acc_82.4_epoch_161.pth \
    --yolo-model ../models/checkpoints/yolo26n.pt \
    --fps 30 \
    --rgb-res 640x480 \
    --depth-res 640x480

## 3. Unit: IMU Calibration
Test IMU connectivity and calibration GUI.
**Expectation:** Independent IMU GUI opens.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
colcon build --packages-select boxbunny_imu --symlink-install
source install/setup.bash
ros2 launch boxbunny_bringup imu_only.launch.py

## 4. Unit: Color Tracking (Simple)
Test **basic punch detection** using HSV color tracking (no heavy AI).
**Expectation:** Launches Camera + Glove Tracker. Publishes `PunchEvent` messages when colored gloves move fast.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
# Launches RealSense Camera + Glove Tracker Node
ros2 launch boxbunny_bringup vision_only.launch.py

## 5. Integration: Reaction Drill (Full System)
Verify the complete user story: Drill Manager + Camera/AI + GUI.
**Workflow:**
1. Run this cell.
2. Wait for Camera window and GUI to open.
3. In GUI -> **Reaction Drill** -> **START**.
4. React to cues. (Uses **Action Model** by default)

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
# 1. Start Drill Manager
ros2 run boxbunny_drills reaction_drill_manager &
PID_DRILL=$!
echo "Started Drill Manager ($PID_DRILL)"
sleep 2

# 2. Start Camera/AI (Action Model)
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/action_prediction
python tools/inference/live_infer_rgbd.py \
    --model-config configs/rgbd_boxing_anticipation.py \
    --model-checkpoint /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/models/action_prediction_model/best_acc_82.4_epoch_161.pth \
    --yolo-model ../models/checkpoints/yolo26n.pt \
    --fps 30 \
    --rgb-res 640x480 \
    --depth-res 640x480 &
PID_CAMERA=$!
echo "Started Camera ($PID_CAMERA)"
sleep 8

# 3. Start GUI
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/src/boxbunny_gui/boxbunny_gui
python gui_main.py

# Cleanup
echo "Stopping processes..."
kill $PID_DRILL $PID_CAMERA || true